In [ ]:
def train(data_train, labels_train):
    #data_train (.npy file) will be used to train model with test and val sets
    #labels_train (.npy file) will be used to test model with labels for test and val sets
    
    #loading training data
    import numpy as np
    import gc
    data_train = np.load(data_train)
    
    #reshaping array to unflatten it to a multidimensional array of size (# of images,imagesize,imagesize,3)
    total_photos=data_train.shape[1]
    img_size = 300
    reshaped_train = np.zeros((total_photos,img_size,img_size,3))

    for i in range(total_photos):
       reshaped_train[i] = np.reshape(data_train[:,0],(1,img_size,img_size,3))
       if i == total_photos - 1:
            del data_train
            gc.collect()
       else:
          data_train = data_train[:,1:]
    reshaped_train = reshaped_train.astype('int16') 
    
    
    #resize to a smaller image
    import cv2

    new_size = 224
    resized_train = np.zeros((reshaped_train.shape[0],new_size,new_size,3))

    for i in range(total_photos):
        resized_train[i] = cv2.resize(reshaped_train[i],dsize = (new_size,new_size),interpolation = cv2.INTER_CUBIC)

    del reshaped_train
    gc.collect()
    
    
    #split between training and validation for model
    from sklearn.model_selection import train_test_split
    labels_train = np.load(labels_train)

    x_train, x_test, y_train, y_test = train_test_split(resized_train, 
                                                                    labels_train,
                                                                    random_state = 0,
                                                                    test_size = 0.2,
                                                                    stratify = labels_train)
    del resized_train
    gc.collect()
    
    
    #converting to 32bit float type
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    #tensorflow libraries
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, BatchNormalization , Dropout, Rescaling, GlobalAveragePooling2D
    
    
    #creating model
    #hyperparameters
    dropout_rate = 0.8
    learning_rate = 1e-4
    
    #transfer learning model
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet',
                                            input_shape=(224, 224, 3),
                                            include_top=False)                
    #all weights are trainable
    base_model.trainable = True
    
    #creating sequential model
    model = Sequential()
    
    #scaling between -1 and 1
    model.add(Rescaling(1./127.5, offset=-1))
    
    #adding transfer learning model
    model.add(base_model)
    
    #GlobalAveragePooling layer
    model.add(GlobalAveragePooling2D())
    
    #Dropout layer
    model.add(Dropout(dropout_rate))
    
    #BatchNormalize layer
    model.add(BatchNormalization())
    
    #Output layer with softmax **change for extra credit
    model.add(Dense(10, activation = 'Softmax'))

    #compiling model with Nadam
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate = learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    #callback will stop after val_loss does not go down in 20 epochs
    #will save best performing model
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                 patience = 20,
                                                 restore_best_weights = True)

    #training model
    history = model.fit(x_train,
                        y_train,
                        epochs=100,
                        callbacks = [callbacks],
                        validation_data=(x_test, y_test))
    
    #save model
    model.save("logo_classification_model2.h5")

In [ ]:
#train model here if needed
train(#, #)